In [2]:
print(not "0")

False


In [5]:
print(set([{"A":1}]))

TypeError: unhashable type: 'dict'

In [ ]:
import os
configs=[]#List of configs to give to PSG later
nSamples=100
atmosphereType="A1"
configFolder=r"C:\Users\Tristan\Downloads\HyPCAR3\configs"
for i,starSample in enumerate([[i]*nSamples for i in range(1,4)]):
    for counter,sample in enumerate(starSample):
        configNum=(i*nSamples)+counter+1#+1 because counter starts at 0
        configFileName=os.path.join(configFolder,f"{atmosphereType}_{configNum}.txt")
        
        print(configFileName)

In [14]:
from pyDOE import lhs
import numpy as np
gStarParamRanges = {
'starRad': (0.8, 1.3),
'starTemp': (5000, 6000),
'Kappa': (-3.5, -2.0),
'Gamma1': (-1.5,  1.1),
'Gamma2': (-1.5,  0.),
'alpha': ( 0.,   1.),
'Albedo':(0.1, 0.8),
'Distance': (1.3,15.)}
lhsSamples=lhs(len(gStarParamRanges),20000)#A list of 8 values 5 times
# scaledSamples=np.zeros_like(lhsSamples)
# print(scaledSamples)

In [2]:
import os
# configFileName=os.path.join(configFolder,f"{atmosphereType}_{configNum}.txt")
print(os.path.join("/home/tristanb/scratch/configFiles", f"A1_{12}.txt"))

/home/tristanb/scratch/configFiles\A1_12.txt


In [2]:
print(20000%32)

0


In [8]:
import numpy as np
def calculateMoleculeAbundances():
    knownValues=np.random.uniform(0,0.001,size=7)#Known moleecule abundancese
    totalKnown=np.sum(knownValues)
    remainingAbundance=1-totalKnown

    backgroundAbundances=np.random.dirichlet(np.ones(3))*remainingAbundance
    print(knownValues)
    print(backgroundAbundances)
    print(list(knownValues)+list(backgroundAbundances))
calculateMoleculeAbundances()

[4.59424601e-04 7.97090207e-04 9.61223096e-05 7.29339462e-04
 4.31713947e-04 9.31741703e-04 5.35175233e-04]
[0.58825567 0.11706405 0.29069968]
[0.0004594246007181554, 0.0007970902069920539, 9.612230962150569e-05, 0.0007293394623348403, 0.00043171394712927694, 0.0009317417027060772, 0.0005351752327408323, 0.5882556661946974, 0.11706405042602595, 0.29069967591703405]


: 

In [7]:
from scipy.stats import lognorm,norm,spearmanr, pearsonr
molecules=["O2","N2","H2","CO2","H2O","CH4","NH3"]
'''
Contains H2O, CH4, NH3, H2/N2
        Lacks CO2, O2
'''
detectionOutput=[0.3,0.6,0.4,0.1,0.7,0.8,0.9]
predAbun=[1,2,2,0,4,5,6]
# predAbun=[0.45,0.001,0.1,0.001,0.30,0.001,0.2]

spearman,p=spearmanr(detectionOutput,predAbun)
pearson,p2=pearsonr(detectionOutput,predAbun)
print((spearman,pearson))
detectionLoss=(1-spearman)+(1-pearson)
print(detectionLoss)

(0.9910312089651149, 0.9617780776627134)
0.04719071337217173


In [3]:
test=[1000,0.43,321]
total=sum(test)
temp=[val/total for val in test]


1.0


In [18]:
import torch
import numpy as np
def calculateLikelihood(yReal,ySim,sigma):
    '''
    This function calculates the likelhood P(A_pred|Y_real). It is a gaussian likelihood function.

    Inputs
    ------
    yReal: Real, input transmittance data
    ySim: Simulated transmittance data, calculated using the predicted abundances
    sigma: Uncertainty of how well predicted abundances translate to simulated spectra

    Returns
    -------
    likelihood: Likelihood value for given real and simulated data
    '''

    mse = torch.mean((yReal - ySim)**2,dim=1)  #Mean squared error between real and simulated data
    #return np.exp(-mse / (2 * sigma ** 2))
    nll = mse / (2 * sigma**2)

    # Optionally include constant term: nll += 0.5 * np.log(2 * np.pi * sigma**2)
    return nll


def classifyAtmosphere(predAbun):
    '''
    This function calculates the H, O, C and N values from the predicted abundances
    Then based on the mixing ratios, we will classify what type of atmosphere it is

    Inputs
    ------
    predAbun: The predicted abundances. A list of 7 values, representing the abundance of each molecule

    
    Returns
    -------
    atmosphereType: A value from the list [A1,A2,B,C]
    '''
    #The order of the molecules
    molecules={0:"O2",1:"N2",2:"H2",3:"CO2",4:"H2O",5:"CH4",6:"NH3"}

    #H = 2*H2 + 2*H2O + 3*NH3 + 4*CH4
    abundanceDict={}
    for i,val in enumerate(predAbun):
        abundanceDict[molecules[i]]=val

    H=2*abundanceDict["H2"] + 2*abundanceDict["H2O"] + 3*abundanceDict["NH3"] + 4*abundanceDict["CH4"]

    C=abundanceDict["CO2"] + abundanceDict["CH4"]

    O=2*abundanceDict["O2"] + 2*abundanceDict["CO2"] + abundanceDict["H2O"]

    N=2*abundanceDict["N2"] + abundanceDict["NH3"]

    if H> 2*O + 4*C:
        if 3*N < H - 2*O - 4*C:
            return "A1"
        else:
            return "A2"
        
    elif 2*O > H + 4*C:
        return "B"
    
    elif abs(H + C + O + N - 1) < 1e-3:  # Hydrogen-poor constraint
        return "C"

    else:
        return "Unkown"


def calculateExpectedValues(predAbun):
    #First figure out what type of atmosphere the model think it is
    #Either A1, A2, B, C or unkown
    atmosphereType=classifyAtmosphere(predAbun)

    molecules={0:"O2",1:"N2",2:"H2",3:"CO2",4:"H2O",5:"CH4",6:"NH3"}
    abundanceDict={}
    for i,val in enumerate(predAbun):
        abundanceDict[molecules[i]]=val

    H=2*abundanceDict["H2"] + 2*abundanceDict["H2O"] + 3*abundanceDict["NH3"] + 4*abundanceDict["CH4"]

    C=abundanceDict["CO2"] + abundanceDict["CH4"]

    O=2*abundanceDict["O2"] + 2*abundanceDict["CO2"] + abundanceDict["H2O"]

    N=2*abundanceDict["N2"] + abundanceDict["NH3"]

    expected={}
    if atmosphereType=="A1":
        #For Type A1 atmospheres (H-rich, mainly H2O, CH4, NH3, and H2; lacking CO2, O2)
        '''
        H > 2O + 4C
        3N < H - 2O -4C

        D = H - N - 2C
        H2O = 2O / D
        NH3 = 2N / D
        CH4 = 2C / D
        H2 = (H - 2O - 4C - 3N) / D
        '''
        D = H - N - 2*C
        expected["H2O"]=2*O / D
        expected["NH3"]=2*N / D
        expected["CH4"]=2*C / D
        expected["H2"]=(H - 2*O - 4*C - 3*N) / D

        expected["O2"]=0
        expected["N2"]=0
        expected["CO2"]=0

    elif atmosphereType=="A2":
        #For Type A2 atmospheres (H-rich, but mainly H2O, CH4, NH3, and N2; lacking CO2, O2)
        '''
        D = H + 2C + 3N + 4O
        H2O = 6O / D
        NH3 = (2H - 8C - 4O) / D
        CH4 = 6C / D
        N2 = (3N + 4C + 2O - H) / D
        '''
        D = H + 2*C + 3*N + 4*O
        expected["H2O"]=6*O / D
        expected["NH3"]=(2*H - 8*C - 4*O) / D
        expected["CH4"]=6*C / D
        expected["N2"] =(3*N + 4*C + 2*O - H) / D

        expected["O2"]=0
        expected["H2"]=0
        expected["CO2"]=0

    elif atmosphereType=="B":
        #For Type B atmospheres (O-rich, mainly O2, N2, CO2, H2O; lacking NH3, H2)
        '''
        D = H + 2O + 2N
        H2O = 2H / D
        N2 = 2N / D
        CO2 = 4C / D
        O2 = (2O - H - 4C) /D
        '''
        D = H + 2*O + 2*N
        expected["H2O"]=2*H / D
        expected["N2"]=2*N / D
        expected["CO2"]=4*C / D
        expected["O2"]=(2*O - H - 4*C) / D

        expected["NH3"]=0
        expected["H2"]=0
        expected["CH4"]=0
    elif atmosphereType=="C":
        #For Type C atmospheres (Hydrogen-poor: mainly H2O, CO2, CH4, N2; lacking NH3, H2, O2)
        #Note: In Type C, the elemental budget is H + C + O + N = 1.
        '''
        Side conditions, so no negative results
        O > 0.5H + 2C -> O2-rich with no CH4
        H > 2O + 4C -> H2 ->H2-rich with no CO2
        C > 0.25H + 0.5O -> graphite condensation with no H2O

        H2O = (H + 2O - 4C) / (H + 2O + 2N)
        CH4 = (H - 2O + 4C) / (2H + 4O + 4N)
        CO2 = (2O + 4C - H) / (2H + 4O + 4N)
        N2 = 2N / (H + 2O + 2N)
        '''

        #Denom for H2O and N2
        D1 = H + 2*O + 2*N

        #Denom for CH4 and CO2
        D2 = 2*H + 4*O + 4*N  

        expected["H2O"] = (H + 2*O - 4*C) / D1
        expected["CH4"] = (H - 2*O + 4*C) / D2
        expected["CO2"] = (2*O + 4*C - H) / D2
        expected["N2"] = 2*N / D1

        expected["O2"] = 0
        expected["H2"] = 0
        expected["NH3"] = 0
    else:
        #Its unkown, prior should just be 1
        #So there is essentially no prior. Just want the likelihood
        return None
    return list(expected.values())

def calculatePrior(predAbun,sigmaPrior):
    '''
    predAbun: The predicted abundance for each moleecule
    sigmaPrior: The uncertainty for each molecule
    '''
    totalLoss=0.0
    batchSize=predAbun.shape[0]
    
    for i in range(batchSize):
        sampleLoss=0.0
        sampleAbundance=predAbun[i]
        sampleSigma=sigmaPrior[i]
        expectedValues=calculateExpectedValues(sampleAbundance.tolist())
        if expectedValues==None:
            totalLoss+=0
        else:
            for j in range(len(sampleAbundance)):
                mu=expectedValues[j]
                sigma=sampleSigma[j]

                y=sampleAbundance[j]
                sampleLoss+=np.log(np.sqrt(2 * np.pi) * sigma) + ((y - mu)**2) / (2 * sigma**2)
            totalLoss+=sampleLoss
    return totalLoss/batchSize



def calculatePosterior(yReal,ySim,sigmaLikelihood,predAbun,sigmaPrior):
    # Math: P({y_{real}}|A_{pred}) \propto P(A_{pred}|Y_{real}) * P(A_{pred})
    '''
    This function calculates the unnormalized posterior

    Inputs
    ------
    yReal: Real, input transmittance data
    ySim: Simulated transmittance data, calculated using the predicted abundances
    sigmaLikelihood: Uncertainty of how well predicted abundances translate to simulated spectra
    predAbun: Predicted abundances
    sigmaPrior: Uncertaintiy of the precicted abundances

    Returns
    -------
    posterior: Unnormalized posterior probability
    '''




    prior=calculatePrior(predAbun,sigmaPrior) 
    likelihood=calculateLikelihood(yReal,ySim,sigmaLikelihood)


    #For the likelihood, I should just take the aggregated transmittance. However, later if I want to include wavelength-molecule mapping, here is where I would do it.

    print(f"Likelihood: {likelihood}")
    print(f"Prior: {prior}")

    posterior=likelihood+prior
    return posterior

yReal=torch.tensor([[1,1,1,0.5,1,1,1],[1,1,1,0.5,1,1,1]])
ySim=torch.tensor([[0.8,0.8,0.8,0.5,0.9,0.75,1],[1,1,1,0.5,1,1,1]])
# predAbun=torch.tensor(np.random.dirichlet(np.ones(7),size=2))

print(predAbun)
print(calculatePosterior(yReal,ySim,0.1,predAbun,torch.tensor([[0.5]*7]*2)))

tensor([[0.1215, 0.1568, 0.2658, 0.2841, 0.0345, 0.1109, 0.0265],
        [0.4767, 0.0158, 0.0696, 0.0232, 0.0657, 0.0773, 0.2717]],
       dtype=torch.float64)
Likelihood: tensor([1.3750, 0.0000])
Prior: 0.9349404987119269
tensor([2.3099, 0.9349])
